# SVM 과제

#### 개요
- EDA, feature engineering (변수선택, scaling 등) (생략해도 되고, Kaggle kernel 참고해도 됨) 
- PCA를 적용해서 차원을 축소 (차원축소의 성능이 안좋으면 원래 데이터 사용가능) 
- SVM 학습, 하이퍼파라미터 튜닝

#### 데이터 출처
Santander data (https://www.kaggle.com/c/santander-customer-transaction-prediction/kernels)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

* 제출양식: {id_code:target}

In [2]:
train_data = pd.read_csv('../data/santader/train.csv')
train_data.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [3]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 202 entries, ID_code to var_199
dtypes: float64(200), int64(1), object(1)
memory usage: 308.2+ MB


In [4]:
train_data.target.value_counts()

0    179902
1     20098
Name: target, dtype: int64

In [5]:
train_data.isnull().sum()

ID_code    0
target     0
var_0      0
var_1      0
var_2      0
          ..
var_195    0
var_196    0
var_197    0
var_198    0
var_199    0
Length: 202, dtype: int64

In [6]:
train_data.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


* 학습데이터 200000개가 있고 학습에 쓰일 feature는 200개입니다. 모두 실수형 데이터들로 이루어져 있고, 결측치는 없습니다.
* train set과 test set으로 나누고 학습하겠습니다.

## 정규화, PCA

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
train_set, test_set = train_test_split(train_data, test_size=0.2, random_state=1)

# 정규화
sc = MinMaxScaler()
train_x = train_set.iloc[:,2:]
train_x = pd.DataFrame(sc.fit_transform(train_x), columns=train_x.columns.values, index=train_x.index.values)
train_y = train_set['target']

test_x = test_set.iloc[:,2:]
test_x = pd.DataFrame(sc.transform(test_x), columns=test_x.columns.values, index=test_x.index.values)
test_y = test_set['target']
print(train_x.shape)
train_x.head()

(160000, 200)


,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
34297,0.431666,0.435672,0.649216,0.754957,0.410017,0.670787,0.415877,0.521477,0.325675,0.506869,...,0.570670,0.343863,0.430076,0.433571,0.660742,0.454683,0.596684,0.445959,0.555825,0.692095
148120,0.341189,0.458413,0.670633,0.348181,0.570038,0.529879,0.309693,0.649003,0.788636,0.629514,...,0.269347,0.549496,0.599866,0.421334,0.487156,0.477811,0.467681,0.821562,0.404377,0.602356
16583,0.771938,0.685973,0.407415,0.424087,0.312221,0.300651,0.713689,0.478290,0.302912,0.718067,...,0.743983,0.847881,0.527180,0.671098,0.382196,0.470657,0.279560,0.673376,0.414625,0.443810
197713,0.586834,0.804726,0.330308,0.656146,0.662755,0.538023,0.299949,0.723307,0.415587,0.779419,...,0.689557,0.356910,0.327448,0.353006,0.200333,0.597940,0.274038,0.470019,0.651296,0.628980
193997,0.707366,0.594895,0.501897,0.736165,0.392339,0.377143,0.545903,0.842374,0.698985,0.219736,...,0.735810,0.389517,0.381070,0.446290,0.303356,0.799914,0.603456,0.632917,0.348214,0.452845


In [12]:
cov_mat = np.cov(train_x.T) # 공분산 행렬을 생성, feature들이 행 쪽에 있어야 해서 T를 해줌
eigen_vals, eigen_vectors = np.linalg.eig(cov_mat) # eig는 eigen value, vector 두 개를 반환해줌
tot = sum(eigen_vals)
ev_li = [i for i in eigen_vals if i > 1]

from sklearn.decomposition import PCA

# n을 1 이상의 자연수로 정해주면 개수를 정하는 것이고, 0~1사이 소수점은 누적설명률을 만족하는 최소개수를 구함
pca = PCA(n_components=.7)
train_pca = pca.fit_transform(train_x.values)
pca_train = pd.DataFrame(train_pca)
pca_train.head()

[]


,0,1,2,3,4,5,6,7,8,9,...,117,118,119,120,121,122,123,124,125,126
0,0.167592,0.081135,0.047414,0.008779,0.304579,0.190861,0.351183,-0.100898,0.027402,-0.260344,...,0.156135,0.106271,-0.210224,0.113943,0.191998,0.138724,-0.122718,-0.093334,0.023321,-0.055598
1,0.281274,0.068239,-0.169085,0.111616,0.053124,-0.096438,0.004179,-0.315170,0.335406,0.094791,...,-0.223968,-0.150133,0.009578,-0.013355,0.096728,0.082755,-0.025177,-0.010478,0.070648,-0.249475
2,-0.061931,-0.101594,-0.217207,-0.105823,-0.000065,0.065007,-0.068998,0.022227,-0.408676,0.145438,...,0.016056,-0.270726,0.168159,-0.058140,0.171617,0.085425,-0.094969,-0.142722,0.041768,-0.194832
3,0.047176,0.040211,-0.375524,-0.077064,0.072685,-0.207208,0.301066,-0.130348,0.092063,0.159606,...,0.040114,-0.038835,0.127926,-0.211426,-0.138764,0.104421,-0.060416,-0.104900,-0.179433,0.159892
4,0.152902,0.168689,0.206730,0.249504,-0.123513,0.038870,0.328965,0.006593,-0.014071,-0.054474,...,-0.059505,0.157673,0.333399,0.276169,0.011638,0.004280,0.228510,0.069710,0.129661,-0.026687


* eigen value > 1인 component는 없었고, 누적설명률 70>이 되는 지점이 127개일 때 였습니다. 

In [ ]:
from sklearn.svm import SVC

svc = SVC()
predicted = svc.fit(pca_train, train_y).predict(pca_test)
accuracy = accuracy_score(test_y, predicted)
print(f'accuracy: {accuracy:.3}')
print(f'cross val score: {cross_val_score(svc, pca_train, train_y, cv=5).mean(): .3}')

C:\Users\yj\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


* 파라미터들 ( grid search 하고싶었는데 도저히 할 엄두가 나지 않았습니다 )
* kernels : 선형 or 비선형 커널함수 선택
* gammas : 비선형 커널일 때 사용. 감마값이 높일 수록 과적합 위험 커짐
* C : error term에 대한 패널티 파라미터. 구분자를 선택하는 강도를 조절해줌
* degree : 커널이 poly일 때 사용. data를 구분하는 강도 설정

* 데이터 수가 많아서인지 훈련이 끝나질 않아서 코랩이 더 빠를까하여 옮겨서 실행해보았습니다.